In [1]:
import time
import csv
import pickle
import operator
import datetime
import os
from tqdm import tqdm
import pandas as pd

In [3]:
os.listdir()

['.ipynb_checkpoints', 'train-item-views.csv', 'Data_preprocess.ipynb']

In [4]:
df=pd.read_csv("train-item-views.csv")
df.head(2)

,sessionId;userId;itemId;timeframe;eventdate
0,1;NA;81766;526309;2016-05-09
1,1;NA;31331;1031018;2016-05-09


In [5]:
def example(data):
    Y=[]
    with open(data, 'r') as fn:
        for ix,line in enumerate(fn):
            if ix<=5:
                Y.append(line.strip("\n"))
    
    return Y

In [7]:
with open(os.path.join(os.getcwd(),'train-item-views.csv'), 'r') as f, open(os.path.join(os.getcwd(),'diginetica-clicks-withHeader.dat'), 'w') as fn:
    # fn.write('sessionId,userId,itemId,timestamp,eventdate'+'\n')
    for line in f:
        fn.write(line) 
        
example('diginetica-clicks-withHeader.dat')

['sessionId;userId;itemId;timeframe;eventdate',
 '1;NA;81766;526309;2016-05-09',
 '1;NA;31331;1031018;2016-05-09',
 '1;NA;32118;243569;2016-05-09',
 '1;NA;9654;75848;2016-05-09',
 '1;NA;32627;1112408;2016-05-09']

In [10]:
dataset = os.path.join(os.getcwd(),'diginetica-clicks-withHeader.dat')
dataset.split("/")[-1].split("-")[0]

'diginetica'

In [17]:
dataset = os.path.join(os.getcwd(),'diginetica-clicks-withHeader.dat')
data_name=dataset.split("/")[-1].split("-")[0]

preprocess_output_log = dict({'dataset name':  data_name})

with open(dataset, "r") as f:
    total_rows=sum(1 for line in f)
    
with open(dataset, "r") as f:
    if data_name == 'yoochoose':
        reader = csv.DictReader(f, delimiter=',')
    else:
        reader = csv.DictReader(f, delimiter=';')
        
    sess_clicks = {}
    sess_date = {}
    ctr = 0
    curid = -1
    curdate = None
    
    for data in tqdm(reader,total=total_rows,leave=True,position=0):
        sessid = data['sessionId']
        if curdate and not curid == sessid:
            date = ''
            if data_name == 'yoochoose':
                date = time.mktime(time.strptime(curdate[:19], '%Y-%m-%dT%H:%M:%S'))
            else:
                date = time.mktime(time.strptime(curdate, '%Y-%m-%d'))
            sess_date[curid] = date
        curid = sessid
        if data_name == 'yoochoose':
            item = data['itemId']
        else:
            item = data['itemId'], int(data['timeframe'])
        curdate = ''
        if data_name == 'yoochoose':
            curdate = data['timestamp']
        else:
            curdate = data['eventdate']

        if sessid in sess_clicks:
            sess_clicks[sessid] += [item]
        else:
            sess_clicks[sessid] = [item]
        ctr += 1
    date = ''
    if data_name == 'yoochoose':
        date = time.mktime(time.strptime(curdate[:19], '%Y-%m-%dT%H:%M:%S'))
    else:
        date = time.mktime(time.strptime(curdate, '%Y-%m-%d'))
        for i in list(sess_clicks):
            sorted_clicks = sorted(sess_clicks[i], key=operator.itemgetter(1))
            sess_clicks[i] = [c[0] for c in sorted_clicks]
    sess_date[curid] = date

100%|█████████▉| 1235380/1235381 [00:07<00:00, 156898.97it/s]


In [18]:
# Filter out length 1 sessions
len1_sessions_filtered_count = 0 
print("{:<30}{:<20,}".format("Before filtering out", len(sess_clicks)))
for s in list(sess_clicks):
    if len(sess_clicks[s]) == 1:
        len1_sessions_filtered_count += 1
        del sess_clicks[s]
        del sess_date[s]

print("{:<30}{:<20,}".format("After filtering out", len(sess_clicks)))
preprocess_output_log['length 1 sessions filtered'] = len1_sessions_filtered_count

Before filtering out          310,324             
After filtering out           219,630             


In [19]:
preprocess_output_log

{'dataset name': 'diginetica', 'length 1 sessions filtered': 90694}

In [20]:
# Count number of times each item appears
iid_counts = {}
for s in sess_clicks:
    seq = sess_clicks[s]
    for iid in seq:
        if iid in iid_counts:
            iid_counts[iid] += 1
        else:
            iid_counts[iid] = 1

sorted_counts = sorted(iid_counts.items(), key=operator.itemgetter(1))

length = len(sess_clicks)
print("{:<30}{:<20,}".format("Before filtering out", len(sess_clicks)))

for s in list(sess_clicks):
    curseq = sess_clicks[s]
    filseq = list(filter(lambda i: iid_counts[i] >= 5, curseq))
    if len(filseq) < 2:
        del sess_clicks[s]
        del sess_date[s]
    else:
        sess_clicks[s] = filseq
        
print("{:<30}{:<20,}".format("After filtering out", len(sess_clicks)))

Before filtering out          219,630             
After filtering out           204,789             


In [22]:
# Split out test set based on dates
dates = list(sess_date.items())
maxdate = dates[0][1]

for _, date in dates:
    if maxdate < date:
        maxdate = date
preprocess_output_log['max date'] = maxdate

# 7 days for test
if data_name == 'yoochoose':
    splitdate = maxdate - 86400 * 1  #the number of seconds for a day：86400
# elif args.dataset in amex_sets: 
#     splitdate = maxdate - 86400 * amex_splitdate #chosen to be 6 days?
else:
    splitdate = maxdate - 86400 * 7 

print('Splitting date', splitdate)      # Yoochoose: ('Split date', 1411930799.0)
tra_sess = filter(lambda x: x[1] < splitdate, dates)
tes_sess = filter(lambda x: x[1] > splitdate, dates)

# Sort sessions by date
tra_sess = sorted(tra_sess, key=operator.itemgetter(1))     # [(sessionId, timestamp), (), ]
tes_sess = sorted(tes_sess, key=operator.itemgetter(1))     # [(sessionId, timestamp), (), ]
print(len(tra_sess))    # 186670    # 7966257
print(len(tes_sess))    # 15979     # 15324
print(tra_sess[:3])
print(tes_sess[:3])

Splitting date 1464134400.0
186670
15979
[('4737', 1451606400.0), ('4741', 1451606400.0), ('4742', 1451606400.0)]
[('289', 1464220800.0), ('290', 1464220800.0), ('302', 1464220800.0)]


In [23]:
preprocess_output_log['len training sessions (tra_sess)'] = len(tra_sess)
preprocess_output_log['len test sessions (tes_sess)'] = len(tes_sess)
preprocess_output_log

{'dataset name': 'diginetica',
 'length 1 sessions filtered': 90694,
 'max date': 1464739200.0,
 'len training sessions (tra_sess)': 186670,
 'len test sessions (tes_sess)': 15979}

In [24]:
# Choosing item count >=5 gives approximately the same number of items as reported in paper
item_dict = {}
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_ids = []
    train_seqs = []
    train_dates = []
    item_ctr = 1
    for s, date in tqdm(tra_sess,total=len(tra_sess),leave=True,position=0):
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_ids += [s]
        train_dates += [date]
        train_seqs += [outseq]
    print(item_ctr)     # 43098, 37484
    preprocess_output_log['item count:'] = item_ctr
    return train_ids, train_dates, train_seqs

# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_ids = []
    test_seqs = []
    test_dates = []
    for s, date in tqdm(tes_sess,total=len(tes_sess),leave=True,position=0):
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_ids += [s]
        test_dates += [date]
        test_seqs += [outseq]
    return test_ids, test_dates, test_seqs


tra_ids, tra_dates, tra_seqs = obtian_tra()
tes_ids, tes_dates, tes_seqs = obtian_tes()


100%|██████████| 186670/186670 [00:00<00:00, 219401.32it/s]


43098


100%|██████████| 15979/15979 [00:00<00:00, 289029.69it/s]


In [26]:
def process_seqs(iseqs, idates):
    out_seqs = []
    out_dates = []
    labs = []
    ids = []
    for id, seq, date in zip(range(len(iseqs)), iseqs, idates):
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
            out_dates += [date]
            ids += [id]
    return out_seqs, out_dates, labs, ids


tr_seqs, tr_dates, tr_labs, tr_ids = process_seqs(tra_seqs, tra_dates)
te_seqs, te_dates, te_labs, te_ids = process_seqs(tes_seqs, tes_dates)
tra = (tr_seqs, tr_labs)
tes = (te_seqs, te_labs)
print(len(tr_seqs))
print(len(te_seqs))
print(tr_seqs[:3], tr_dates[:3], tr_labs[:3])
print(te_seqs[:3], te_dates[:3], te_labs[:3])

719470
60858
[[1], [3, 4], [3]] [1451606400.0, 1451606400.0, 1451606400.0] [2, 5, 4]
[[21553, 20071, 8762, 21566, 6381], [21553, 20071, 8762, 21566], [21553, 20071, 8762]] [1464220800.0, 1464220800.0, 1464220800.0] [21566, 6381, 21566]


In [27]:
all = 0

preprocess_output_log['len training seqs (tr_seqs)'] = len(tr_seqs)
preprocess_output_log['len test seqs (te_seqs)'] = len(tr_seqs)

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('avg length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

preprocess_output_log['avg length'] = all/(len(tra_seqs) + len(tes_seqs) * 1.0)


avg length:  4.850942344040704


In [28]:
preprocess_output_log

{'dataset name': 'diginetica',
 'length 1 sessions filtered': 90694,
 'max date': 1464739200.0,
 'len training sessions (tra_sess)': 186670,
 'len test sessions (tes_sess)': 15979,
 'item count:': 43098,
 'len training seqs (tr_seqs)': 719470,
 'len test seqs (te_seqs)': 719470,
 'avg length': 4.850942344040704}

In [30]:
pickle.dump(tra, open('train.txt', 'wb'))
pickle.dump(tes, open('test.txt', 'wb'))
pickle.dump(tra_seqs, open('all_train_seq.txt', 'wb'))
pickle.dump(tes_seqs, open('all_test_seq.txt', 'wb'))